# panelby
Julian Hsu
This is our package for panel models.

In [179]:
import pandas as pd
import numpy as np
import os as os 

import matplotlib.pyplot as plt
%matplotlib inline  
from IPython.display import display    

import scipy.stats 
from sklearn.linear_model import ElasticNet
import statsmodels.api as sm

from typing import List
from operator import add
from toolz import reduce, partial
from scipy.optimize import minimize

In [180]:
## Create the univeral function that calls all the different SC models coded up below.
## It will also call functions used for SC model validation

class sc:
    def sc_model(model_name='adh',
                data=None,
                data_dict= {'treatment':None, 'date':None, 'post':None, 'unitid':None, 'outcome':None},
                pre_process_data=None,
                pre_treatment_window=None):
        ## First pre-process the data if possible.
        if pre_process_data==None:
            pre_process_data = dgp.clean_and_input_data(dataset=data,
                                                        treatment=data_dict['treatment'],
                                                        unit_id=data_dict['unitid'],
                                                        date=data_dict['date'],
                                                        post=data_dict['post'],
                                                        outcome=data_dict['outcome'])
        else:
            pass
        
        ## Second calculate the pre-treatment-window that can be useful for placebo tests
        pre_treatment_window = dgp.determine_pre_treatment_window(ci_data_output=pre_process_data,
                                                                 pre_treatment_window=pre_treatment_window)
        
        
        ## Finally, call an SC model.
        if model_name=='adh':
            print('Using ADH')
            sc_est = adh.predict_omega(pre_process_data['T_pre'], 
                                    pre_process_data['C_pre'], 
                                    pre_treatment_window)
            sc_output = di.sc_style_results(pre_process_data['T_pre'], 
                                             pre_process_data['T_pst'],
                                             pre_process_data['C_pre'], 
                                             pre_process_data['C_pst'],
                                np.zeros(pre_process_data['T_pst'].shape[1]), np.array(sc_est['omega']))
            sc_est['mu'] = np.zeros( pre_process_data['T_pre'].shape[1] )
            
        elif model_name=='di':
            print('Using DI')
            w=alpha_lambda.get_alpha_lambda(sc_dict['C_pre'])
            alpha_lambda_to_use = alpha_lambda.alpha_lambda_transform(w.x)
            ## Take the alpha and lambda values, and estimate mu and omega
            sc_est = di.predict_mu_omega(pre_process_data['T_pre'], 
                                         pre_process_data['C_pre'], alpha_lambda_to_use, 
                                         pre_treatment_window)
            sc_output = di.sc_style_results(pre_process_data['T_pre'], 
                                            pre_process_data['T_pst'],
                                            pre_process_data['C_pre'], 
                                            pre_process_data['C_pst'],
                                            sc_est['mu'],sc_est['omega'])
            
        elif model_name=='cl':
            print('Using CL')
            sc_est = cl.predict_mu_omega(pre_process_data['T_pre'],
                                      pre_process_data['C_pre'], 
                                      pre_treatment_window)
            sc_output = di.sc_style_results(pre_process_data['T_pre'],
                                            pre_process_data['T_pst'],
                                            pre_process_data['C_pre'], 
                                            pre_process_data['C_pst'],
                                            sc_est['mu'], sc_est['omega']) 
        else:
            print('SC Model name not supported. \n [adh,di,cl] are supported models.')
        
        ## Output measures of fit pre-treatment (training), pre_treatment (test), and post-treatment
        sc_validation = sc.sc_validation(treatment_pre=pre_process_data['T_pre'], 
                     treatment_pst=pre_process_data['T_pst'],
                     control_pre=  pre_process_data['C_pre'],
                     control_pst=  pre_process_data['C_pst'], 
                     mu=  sc_est['mu'],
                     omega=sc_est['omega'],
                     pre_treatment_window=pre_treatment_window)
        
        ## Improve on this by calling for inference results
        sc_df_results = sc.collect_sc_outputs(sc_output = sc_output,
                           pre_process_data=pre_process_data,
                       theta_grid = np.arange(-10,10,0.5),
                           pre_treatment_window=pre_treatment_window, alpha = 0.05)
        
        return {**sc_output, 'results_df':sc_df_results}


    '''
    Create a function that evaluates how well the SC model does at matching the:
    1. pre-treatment data used for training;
    2. pre-treatment data used for testing; and
    3. the post-treatment data.
    
    Calculate metrics of fit and compare pre-treatment and post-treatment metrics.
    '''
    def sc_validation(treatment_pre, treatment_pst, control_pre, control_pst, 
                                      mu,omega,
                                     pre_treatment_window):
        ## Re-combine the observed treatment and control outcomes
        y_treat_obs = pd.concat([treatment_pre, treatment_pst], axis=0)
        y_control_obs = pd.concat([control_pre, control_pst], axis=0)

        ## Estimate the counterfactual outcome of the treatment group
        y_treat_hat = mu + np.dot(y_control_obs, omega.T)

        from sklearn.metrics import mean_absolute_percentage_error
        def comparison_over_windows(x,y,
                                    pre_treatment_window,
                                   metric_func, index_name):
            x_pre_train = x[0:pre_treatment_window[0]]
            y_pre_train = y[0:pre_treatment_window[0]]
            x_pre_test = x[pre_treatment_window[0]:pre_treatment_window[0]+pre_treatment_window[1]]
            y_pre_test = y[pre_treatment_window[0]:pre_treatment_window[0]+pre_treatment_window[1]]
            x_pst_test = x[-1*(pre_treatment_window[0]+pre_treatment_window[1]):]
            y_pst_test = y[-1*(pre_treatment_window[0]+pre_treatment_window[1]):]
            test_pre_train = metric_func(x_pre_train, y_pre_train)
            test_pre_test  = metric_func(x_pre_test,  y_pre_test)
            test_pst_test  = metric_func(x_pst_test,  y_pst_test)
            
            return pd.DataFrame(index=[index_name], data={'test_pre_train':test_pre_train,
                   'test_pre_test':test_pre_test,
                   'test_pst_test':test_pst_test})

        ## Compare the predicted treatment with the observed treatment
        treat_hat_treat_obs = comparison_over_windows(y_treat_hat, y_treat_obs,
                                                   pre_treatment_window,
                                                   mean_absolute_percentage_error,'mape_vs_treat_obs')
        return treat_hat_treat_obs

    def sc_validation_gather(counterfactual=None,
                              actual=None,
                             pre_treatment_window=None):
        from sklearn.metrics import mean_absolute_percentage_error        

        x_pre_train = counterfactual[0:pre_treatment_window[0]]
        y_pre_train = actual[0:pre_treatment_window[0]]
        x_pre_test = counterfactual[pre_treatment_window[0]:pre_treatment_window[0]+pre_treatment_window[1]]
        y_pre_test = actual[pre_treatment_window[0]:pre_treatment_window[0]+pre_treatment_window[1]]
        x_pst_test = counterfactual[-1*(pre_treatment_window[0]+pre_treatment_window[1]):]
        y_pst_test = actual[-1*(pre_treatment_window[0]+pre_treatment_window[1]):]
        test_pre_train = mean_absolute_percentage_error(x_pre_train, y_pre_train)
        test_pre_test  = mean_absolute_percentage_error(x_pre_test,  y_pre_test)
        test_pst_test  = mean_absolute_percentage_error(x_pst_test,  y_pst_test)
            
        return pd.DataFrame(index=['mape_test'], data={'test_pre_train':test_pre_train,
               'test_pre_test':test_pre_test,
               'test_pst_test':test_pst_test})

    '''
    Output a dataframe collecting different metrics for each treated unit,
    such as the ATET, p-value, confidence interval, and placebo tests.
    
    All confidence intervals and p-values to be calculated for individual post-treatment periods
    '''
    def collect_sc_outputs(sc_output = None,
                           pre_process_data=None,
                           theta_grid = None,  # np.arange(-10,10,0.5),
                           pre_treatment_window=None,
                           aggregate_pst_periods=True,
                      alpha = 0.05):
        if aggregate_pst_periods==True:
            a = sc.collect_sc_outputs_aggregate(sc_output = sc_output,
                           pre_process_data=pre_process_data,
                           theta_grid = theta_grid,  # np.arange(-10,10,0.5),
                           pre_treatment_window=pre_treatment_window,
                      alpha = 0.05)
        else:
            a = sc.collect_sc_outputs_individual(sc_output = sc_output,
               pre_process_data=pre_process_data,
               theta_grid = theta_grid,  # np.arange(-10,10,0.5),
               pre_treatment_window=pre_treatment_window,
              alpha = 0.05)
        return a
    def collect_sc_outputs_individual(sc_output = None,
                           pre_process_data=None,
                           theta_grid = None,  # np.arange(-10,10,0.5),
                           pre_treatment_window=None,
                           aggregate_pst_periods=True,
                      alpha = 0.05):
        ## To do the individual results, just call the function that does that aggregated
        ## results multiple times, assuming that there is only one post-treatment period.
        
        ## Calculate some primatives:
        pre_T = pre_process_data['T_pre'].shape[0]
        pst_T = pre_process_data['T_pst'].shape[0]
        permutations_subset_block_individual = []
        for i in range(pre_T+1):
            half_A = time_list[-1*(pre_T-i):]
            half_B = time_list[0:i]
            scrambled_list = np.concatenate([half_A, half_B]) 
            permutations_subset_block_individual.append( list(scrambled_list)  )
        
        collect_individual_df = pd.DataFrame()
        for t_pst in range(1, pst_T+1):
            pst_index = np.append(np.arange(pre_periods) ,[pre_T+t_pst]) 
            sc_output_subset = {'atet':sc_output['atet'].iloc[pst_index],
                               'predict_est':sc_output['predict_est'].iloc[pst_index],}
            pre_process_data_subset = {'time_scramble': permutations_subset_block_individual,
                                      'treatment_window':pre_process_data['treatment_window'][:]}
            a = collect_sc_outputs_aggregate(sc_output = sc_output_subset,
                                       pre_process_data=pre_process_data_subset,
                                       theta_grid = theta_grid,
                                       pre_treatment_window=pre_process_data['treatment_window'],
                                  alpha = 0.05)
            a['individual_post']     = t_pst
            collect_individual_df = pd.concat([collect_individual_df, a])
        return collect_individual_df
    
    def collect_sc_outputs_aggregate(sc_output = None,
                           pre_process_data=None,
                           theta_grid = None,  # np.arange(-10,10,0.5),
                           pre_treatment_window=None,
                      alpha = 0.05): 
        sc_results = sc_output['atet'].mean(axis=0).copy().to_frame().rename(columns={0:'atet'})
            
        sc_pv = []
        sc_ci_05 = []
        sc_ci_95 = []
        sc_se = []
        sc_placebo_pre_train = []
        sc_placebo_pre_test = []
        sc_placebo_pst_test = []
            
        o = 0
        for p in [x for x in sc_output['predict_est'].columns if '_est' not in x]:
            ## Calculate the p-value
            pv_output = conformal_inf.pvalue_calc(counterfactual=np.array( sc_output['predict_est']['{0}_est'.format(p)].tolist() ),
                                      actual=np.array( sc_output['predict_est']['{0}'.format(p)].tolist() ),
                                      permutation_list =pre_process_data['time_scramble'],
                                      treatment_window = pre_process_data['treatment_window'],
                                      h0=0)
            sc_pv.append(pv_output)

            ## Calculate the confidence interval
            ## If no pre-defined theta grid is defined, then just look 100 values in either direction
            ## of the ATET estimate
            if theta_grid is None:
                i = sc_results['atet'].iloc[o]/100
                theta_grid_use = np.arange(-500*i+sc_results['atet'].iloc[o],
                                      -500*i+sc_results['atet'].iloc[o], i*5 )            
            else:
                theta_grid_use = theta_grid[:]
            ci_output = conformal_inf.ci_calc(y_hat=sc_output['predict_est']['{0}_est'.format(p)].values,
                           y_act=sc_output['predict_est']['{0}'.format(p)].values,
                           theta_grid=theta_grid_use,
                           permutation_list_ci =pre_process_data['time_scramble'],
                           treatment_window_ci = pre_process_data['treatment_window'],
                           alpha=alpha)
            sc_ci_05.append(ci_output['ci_interval'][0])
            sc_ci_95.append(ci_output['ci_interval'][1])

            ## Calculate the placebo tests for that treated unit too
            placebo_df = sc.sc_validation_gather(counterfactual=np.array( sc_output['predict_est']['{0}_est'.format(p)].tolist() ),
                                      actual=np.array( sc_output['predict_est']['{0}'.format(p)].tolist() ),
                                     pre_treatment_window=pre_process_data['treatment_window'])
            sc_placebo_pre_train.append(placebo_df['test_pre_train'].values[0])
            sc_placebo_pre_test.append( placebo_df['test_pre_test'].values[0])
            sc_placebo_pst_test.append( placebo_df['test_pst_test'].values[0]) 
            o+=1

        sc_results['pvalues'] = sc_pv
        sc_results['ci_lower'] = sc_ci_05
        sc_results['ci_upper'] = sc_ci_95
        sc_results['alpha'] = alpha
        sc_results['test_pre_train_MAPE'] = sc_placebo_pre_train
        sc_results['test_pre_test_MAPE'] = sc_placebo_pre_test
        sc_results['test_pst_test_MAPE'] = sc_placebo_pst_test
        
        return sc_results

    
## DGP and functions to determine what types of SC models to do, and calculate primatives for SC models
class dgp:    
    def determine_pre_treatment_window(ci_data_output=None,
                                      pre_treatment_window=None):
        if pre_treatment_window ==None:
            pre_treatment_len = ci_data_output['C_pre'].shape[0]
            pre_t0 = int( 0.75*pre_treatment_len )
            if pre_t0 < 1:
                pre_t0=1
            else:
                pass            
            pre_t1 = pre_treatment_len - pre_t0
            pre_treatment_window = [pre_t0, pre_t1]
        else:
            pass
        return pre_treatment_window 
    
    def clean_and_input_data(dataset=None, 
                             treatment='treated_unit', 
                             unit_id = 'unitid',
                             date='T',
                             post='post', outcome='Y'):
        
        C_pre = dataset.loc[(dataset[treatment]==0) & (dataset[post]==0)].pivot_table(columns=unit_id,
                                                index=date,
                                                values=outcome)
        C_pst = dataset.loc[(dataset[treatment]==0) & (dataset[post]==1)].pivot_table(columns=unit_id,
                                                index=date,
                                                values=outcome)
        T_pre = dataset.loc[(dataset[treatment]==1) & (dataset[post]==0)].pivot_table(columns=unit_id,
                                                index=date,
                                                values=outcome)
        T_pst = dataset.loc[(dataset[treatment]==1) & (dataset[post]==1)].pivot_table(columns=unit_id,
                                                index=date,
                                                values=outcome)
        
        permutations_subset_block = conformal_inf.time_block_permutation(data=dataset, 
                                                                         time_unit=date,
                                                                         post=post)
                
        return {'C_pre':C_pre, 'C_pst':C_pst, 'T_pre':T_pre, 'T_pst':T_pst, 
                'time_scramble':permutations_subset_block[0],
               'treatment_window':permutations_subset_block[1]}

In [181]:
## Doudchenko Imbens, (2016) Model
class di:
    def estimate_mu_omega(treatment_pre, control_pre, alpha_lambda_0):
        alpha_0, lambda_0 = alpha_lambda_0[0], alpha_lambda_0[1]
        elnet = ElasticNet(random_state=2736, alpha=alpha_0, l1_ratio=lambda_0)
        elnet.fit(control_pre, treatment_pre )
        ## Output interpretable weights
        try:
            df_weights= pd.DataFrame(data=zip(treatment_pre.columns,
                                              elnet.coef_.T
                                             ))
        except:
            df_weights = pd.DataFrame(index=np.arange(len(elnet.coef_)), 
                         data=elnet.coef_.T)        
        return {'mu': elnet.intercept_, 'omega': elnet.coef_, 'weights':df_weights, 'full':elnet}

    def predict_mu_omega(treatment_pre, control_pre, alpha_lambda_0, holdout_windows):
        ## Don't use all the control data
        ## Make sure that the holdout windows add up to the total number of pre-treatment units
        if (holdout_windows[0]+holdout_windows[1] != len(control_pre)):
            print('the arg holdout_windows does not add up to the number of time units!')
            print('holdout_windows = {0}'.format(holdout_windows))
            print('total number of time periods = {0}'.format(len(control_pre)))
        else:
            pass    
        ## Define the holdout samples
        control_holdout = control_pre[0:holdout_windows[0]]
        treatment_holdout = treatment_pre[0:holdout_windows[0]]    
        
        control_nonholdout = control_pre[holdout_windows[0]:]
        treatment_nonholdout = treatment_pre[holdout_windows[0]:]
        
        ## Estimate the DI model
        holdout_dict = di.estimate_mu_omega(treatment_holdout, control_holdout, alpha_lambda_0)
        if treatment_pre.shape[1]==1:
            holdout_dict['omega'] = np.array([holdout_dict['omega']])
        else:
            pass
        
        ## Estimate measure of fit for the hold out and non-holdout sample
        diff_holdout = treatment_holdout       -\
            np.dot(control_holdout, holdout_dict['omega'].T)+holdout_dict['mu']        
        diff_nonholdout = treatment_nonholdout -\
            np.dot(control_nonholdout, holdout_dict['omega'].T)+holdout_dict['mu']

        diff_nonholdout_mse = (diff_nonholdout**2).mean()
        diff_holdout_mse = (diff_holdout**2).mean()
        return {'mu':     holdout_dict['mu'],
               'omega':   holdout_dict['omega'],
               'weights': holdout_dict['weights'],
               'full':    holdout_dict['full'],
               'mse_holdout': diff_holdout_mse,
               'mse_nonholdout':diff_nonholdout_mse}
    
    def sc_style_results(treatment_pre, treatment_pst, control_pre, control_pst, mu,omega):
        ## Do some normalization of the omega input
        if len(omega.shape) > 2:
            omega = omega.reshape(omega.shape[-2],omega.shape[-1])
        else:
            pass
        
        final_X = pd.concat([treatment_pre, treatment_pst], axis=0)
        control_X = pd.concat([control_pre, control_pst], axis=0)

        control_df = mu + pd.DataFrame(data=np.dot(control_X, omega.T), columns=[ l+'_est' for l in final_X.columns ])
        control_df.index = final_X.index
        
        output_df = control_df.join(final_X)
        
        treatment_periods = -1*len(treatment_pst)
        atet_df = pd.DataFrame()
        for c in [l for l in output_df.columns if '_est' not in l]:
            diff = output_df[c][treatment_periods:] - output_df[c+'_est'][treatment_periods:]
            atet_df[c] = diff
        
        return {'atet': atet_df , 'predict_est':output_df}

In [182]:
## Here code up the functions that we will try to minimize over
class alpha_lambda:
    def diff(y_t,y_c, mu_x, omega_x):
        return y_t - mu_x - np.dot(y_c,omega_x)

    def alpha_lambda_transform(alpha_lambda_raw_):
        ## Alpha is strictly greater than zero
        ## lambda exists between 0 and 1
        return np.exp(alpha_lambda_raw_[0]/1000), np.exp(alpha_lambda_raw_[1])/(1+np.exp(alpha_lambda_raw_[1])), 
    def alpha_lambda_diff(alpha_lambda_raw_, control_pre):
        ## Transform the inputted alpha,lambda values 
        alpha_lambda_t = alpha_lambda.alpha_lambda_transform(alpha_lambda_raw_)
        difference_array = []
        ## Pick one control unit as the pretend treatment unit
        for u in control_pre.columns:
            control_pre_placebo_treat = control_pre[u]
            control_pre_placebo_cntrl = control_pre[ [l for l in control_pre.columns if l !=u] ]

            ## Estimate mu and lambda with that control unit
            control_pre_placebo_w = di.estimate_mu_omega(control_pre_placebo_treat,
                             control_pre_placebo_cntrl,
                             alpha_lambda_t)
            ## Estimate the difference
            d = alpha_lambda.diff(control_pre_placebo_treat, 
                 control_pre_placebo_cntrl, 
                 control_pre_placebo_w['mu'],
                 control_pre_placebo_w['omega'])
            difference_array.append(d)
        ## Estimate the difference across all the control units
        d_mean = np.mean(difference_array)
        return d_mean
    def get_alpha_lambda(control_pre_input):
        ## Initialize at a given point
        weights = minimize(partial(alpha_lambda.alpha_lambda_diff, control_pre=control_pre_input),
                             np.array([10.15,0.5]),
                           method='BFGS',
                          options={'maxiter':5000, 'gtol': 1e-07, 'disp':False})
        return weights

    
    

In [183]:
## Constrained Lasso Model
from scipy.optimize import fmin_slsqp

class cl:
    def cl_obj(params, y,x) -> float:
        return np.mean(  (y - params[0] - np.dot(x,params[1:]))**2)
    
    def predict_mu_omega(treatment_pre, control_pre, holdout_windows):
        ## Don't use all the control data
        ## Make sure that the holdout windows add up to the total number of pre-treatment units
        if (holdout_windows[0]+holdout_windows[1] != len(control_pre)):
            print('the arg holdout_windows does not add up to the number of time units!')
            print('holdout_windows = {0}'.format(holdout_windows))
            print('total number of time periods = {0}'.format(len(control_pre)))
        else:
            pass    
        ## Define the holdout samples
        control_holdout = control_pre[0:holdout_windows[0]]
        treatment_holdout = treatment_pre[0:holdout_windows[0]]    
        
        control_nonholdout = control_pre[holdout_windows[0]:]
        treatment_nonholdout = treatment_pre[holdout_windows[0]:]
        
        ## Estimate the CL model
        ## Let's loop over different treatment units.
        holdout_dict = {}
        holdout_dict['mu'] = []
        holdout_dict['omega'] = []
        holdout_dict['weights'] = []
        diff_holdout_mse = []
        diff_nonholdout_mse = []
        if treatment_pre.shape[1] > 1:
            for t in treatment_pre.columns:
                t_dict = cl.get_mu_omega(treatment_holdout[t], control_holdout)
                ## Estimate measure of fit for the hold out and non-holdout sample
                diff_h = treatment_holdout[t]       - np.dot(control_holdout, t_dict['omega'].T)+t_dict['mu']
                diff_h_mse = (diff_h**2).mean()
                diff_nh = treatment_nonholdout[t] - np.dot(control_nonholdout, t_dict['omega'].T)+t_dict['mu']
                diff_nh_mse = (diff_nh**2).mean()
        
                holdout_dict['mu'].append(t_dict['mu'])
                holdout_dict['omega'].append(t_dict['omega'])
                holdout_dict['weights'].append(t_dict['weights'])
                diff_holdout_mse.append(diff_h_mse)
                diff_nonholdout_mse.append(diff_nh_mse)
        else:
            
            t_dict = cl.get_mu_omega(treatment_holdout.values.flatten(), control_holdout)
            ## Estimate measure of fit for the hold out and non-holdout sample
            t_dict['omega'] = np.array([t_dict['omega']])
            diff_h= treatment_holdout       - np.dot(control_holdout, t_dict['omega'].T)+t_dict['mu']
            diff_h_mse = (diff_h**2).mean()
            diff_nh = treatment_nonholdout - np.dot(control_nonholdout, t_dict['omega'].T)+t_dict['mu']
            diff_nh_mse = (diff_nh**2).mean()

            holdout_dict['mu'].append(t_dict['mu'])
            holdout_dict['omega'].append(t_dict['omega'])
            holdout_dict['weights'].append(t_dict['weights'])
            diff_holdout_mse.append(diff_h_mse)
            diff_nonholdout_mse.append(diff_nh_mse)            
                    
        holdout_dict['omega'] = np.array(holdout_dict['omega'])
        if len(holdout_dict['omega'].shape) > 2:
            holdout_dict['omega'] = holdout_dict['omega'].reshape(holdout_dict['omega'].shape[-2],holdout_dict['omega'].shape[-1])
        else:
            pass        
        holdout_dict['mse_holdout'] =np.mean(diff_holdout_mse)
        holdout_dict['mse_nonholdout'] =np.mean(diff_holdout_mse)        

        return holdout_dict


    def get_mu_omega(treatment_pre_input, control_pre_input):
        n = control_pre_input.shape[1]
        initialx = np.ones(n+1)/1
        ## Initialize at a given point
        weights = fmin_slsqp(partial(cl.cl_obj, y=treatment_pre_input,
                                  x=control_pre_input),
                             initialx,
                             f_ieqcons=lambda x: 1-np.sum(np.abs(x[1:])),
                         iter=50000, 
                         disp=False)
        mu, omega = weights[0], weights[1:]
        return {'mu':mu, 'omega':omega, 'weights':weights}
    

In [184]:
## Abadie, Diamond, Hainmueller (2010) model
## Also code up the ADH weights
## Abadie/Diamond/Hainmueller    
from typing import List
from operator import add
from toolz import reduce, partial
from scipy.optimize import fmin_slsqp

class adh:
    ## Define loss function
    def loss_w(W, X, y) -> float:
        return np.sqrt(np.mean((y - X.dot(W))**2))

    def get_w(X, y):
        ## Initialize at sample average with some noise
        w_start = [1/X.shape[1]]*X.shape[1]
    #     w_start = np.ones(X.shape[1])

        weights = fmin_slsqp(partial(adh.loss_w, X=X, y=y),
                             np.array(w_start),
                             f_eqcons=lambda x: np.sum(x) - 1,
                             iter=50000, 
                             bounds=[(0.0, 1.0)]*len(w_start),
                             disp=False)
        return weights   
    
    def predict_omega(treatment_pre, control_pre, holdout_windows):
        ## Don't use all the control data
        ## Make sure that the holdout windows add up to the total number of pre-treatment units
        if (holdout_windows[0]+holdout_windows[1] != len(control_pre)):
            print('the arg holdout_windows does not add up to the number of time units!')
            print('holdout_windows = {0}'.format(holdout_windows))
            print('total number of time periods = {0}'.format(len(control_pre)))
        else:
            pass    
        ## Define the holdout samples
        control_holdout = control_pre[0:holdout_windows[0]]
        treatment_holdout = treatment_pre[0:holdout_windows[0]]    
        
        control_nonholdout = control_pre[holdout_windows[0]:]
        treatment_nonholdout = treatment_pre[holdout_windows[0]:]
        
        ## Estimate the CL model
        ## Let's loop over different treatment units.
        holdout_dict = {}
        holdout_dict['omega'] = []
        holdout_dict['weights'] = []
        diff_holdout_mse = []
        diff_nonholdout_mse = []
        if treatment_pre.shape[1] > 1:
            for t in treatment_pre.columns:
                t_dict = adh.get_w(control_holdout,treatment_holdout[t])
                ## Estimate measure of fit for the hold out and non-holdout sample
                diff_h = treatment_holdout[t]       - np.dot(control_holdout, t_dict.T)
                diff_h_mse = (diff_h**2).mean()
                diff_nh = treatment_nonholdout[t] - np.dot(control_nonholdout, t_dict.T)
                diff_nh_mse = (diff_nh**2).mean()
        
                holdout_dict['omega'].append(t_dict)
                holdout_dict['weights'].append(t_dict)
                diff_holdout_mse.append(diff_h_mse)
                diff_nonholdout_mse.append(diff_nh_mse)
        else:
            t_dict = adh.get_w(control_holdout,treatment_holdout.values.flatten())
            ## Estimate measure of fit for the hold out and non-holdout sample
            diff_h= treatment_holdout       - np.dot(control_holdout, np.array([t_dict]).T)
            diff_h_mse = (diff_h**2).mean()
            diff_nh = treatment_nonholdout - np.dot(control_nonholdout, np.array([t_dict]).T)
            diff_nh_mse = (diff_nh**2).mean()

            holdout_dict['omega'].append(t_dict)
            holdout_dict['weights'].append(t_dict)
            diff_holdout_mse.append(diff_h_mse)
            diff_nonholdout_mse.append(diff_nh_mse)            
        holdout_dict['omega'] = np.array(holdout_dict['omega'])
        holdout_dict['mse_holdout'] =np.mean(diff_holdout_mse)
        holdout_dict['mse_nonholdout'] =np.mean(diff_holdout_mse)        
        
        return holdout_dict    
    

In [185]:
## Conformal Inference to do inference for the SC models
import itertools
from numpy.random import default_rng

rng = default_rng()
rng.choice(10, size=10, replace=False)


class conformal_inf:
    def time_block_permutation(data=None,
                              time_unit='date',
                              post='W'):
        tw =  len(data.loc[ data[post]==1][time_unit].unique())
        treatment_window = len(data.loc[(data[post]==0)][time_unit].unique())-tw, tw

        time_list = np.arange( np.sum(treatment_window) )
        T_len = len(time_list)

        ## Time block permutations
        permutations_subset_block = []

        for i in range(T_len):
            half_A = time_list[-1*(T_len-i):]
            half_B = time_list[0:i]
            scrambled_list = np.concatenate([half_A, half_B]) 
            permutations_subset_block.append( list(scrambled_list)  )

        return permutations_subset_block, treatment_window


    def scrambled_residual(counterfactual, actual, 
                       scrambled_order,
                      treatment_window):
        '''
        counterfactual   array of counterfactual estimates that are assumed to be ordered sequentially
        actual           ``'' for actual values
        scrambled_order  integer array that tells me how to scramble these
        treatment_window list of two integers for the number of pre-treatment and post-treatment units
        '''
        counterfactual_ = counterfactual[scrambled_order][-1*treatment_window[1]:]
        actual_         = actual[scrambled_order][-1*treatment_window[1]:]    
        return actual_ - counterfactual_
    
    def test_statS(q, treatment_window, residual):
        normed = np.sum(  np.power(np.abs(residual), q)  )
        return np.power( treatment_window[1]**(-0.5)*normed , 1/q)    

    def pvalue_calc(counterfactual=None,
                    actual=None, 
                    permutation_list=None,
                   treatment_window=None,
                   h0 = 0):
        control_pst = counterfactual[-1*treatment_window[1]:]
        actual_pst  = actual[-1*treatment_window[1]:] 
        actual_pst -= h0

        ## Calculate the residual
        residual_initial = np.abs(actual_pst - control_pst)         
        S_q = conformal_inf.test_statS(1, treatment_window, residual_initial)

        ## Now do a whole bunch of treatment time scrambles
        ## We're going to permute over all time-based permutations
        ## Adjust the actual by the null hypothesis 
        treat_ = actual[:]
        treat_[-1*treatment_window[1]:] -= h0
        full_residual = np.abs(treat_ - counterfactual)
        S_q_pi = []                
        
        for r in permutation_list:
            scrambled_dates = np.array(list(r))              
            residual_ = full_residual[scrambled_dates][-1*treatment_window[1]:]
            S_q_pi.append(  conformal_inf.test_statS(1, treatment_window, residual_ )  )
        p_value = 1 - np.average( (np.array(S_q_pi) < S_q ) )
        return p_value
    
    def ci_calc(y_hat=None,
               y_act=None,
               theta_grid=None,
                permutation_list_ci = None,
                treatment_window_ci =None,
               alpha=0.05):
        pv_grid = []
        for t in theta_grid:
            pv = conformal_inf.pvalue_calc(counterfactual=y_hat.copy(),
                        actual=y_act.copy(), 
                            permutation_list = permutation_list_ci,
                           treatment_window = treatment_window_ci,
                                 h0=t)
            pv_grid.append(pv)   
        ci_list = [ theta_grid[i] for i in range(len(pv_grid)) if pv_grid[i] < alpha ]
        if len(ci_list)==0:
            ci_list = [-9999]
        return {'theta_list':theta_grid, 'pvalue_list':pv_grid, 'ci_list':ci_list,
               'ci_interval':[np.min(ci_list), np.max(ci_list)]
               }


Test these functions out.

In [186]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

N = 100
initial = np.random.uniform(0,1, N)
df = pd.DataFrame(data={'y':initial,
                  'unit_id':np.arange(N),
                       'time':np.zeros(N).astype(int)})
for t in range(10):
    entry = df.iloc[-1*N:]
    entry['y'] = entry['y']*0.80 + np.random.normal(0,0.5,N)*0.20
    entry.loc[[True]*N,'time'] = t
    df = pd.concat([df,entry])
df['treated']        = df['unit_id'].isin([0,1,2])
df['post'] = (df['time'] > 7)
df['W'] =     df['treated']*    df['post']
df['unit_id'] = df['unit_id'].apply(str)
df.loc[df['W']==True, 'y'] += 3
## Clean data

sc_dict = dgp.clean_and_input_data(dataset=df,
                                   treatment='treated',
                                   unit_id='unit_id',
                                   date='time',
                                   post='post',
                                  outcome='y')

In [187]:
treatment_window_pre_treatment = dgp.determine_pre_treatment_window(ci_data_output=sc_dict, pre_treatment_window=None)

In [188]:
f3 = sc.sc_model(model_name='adh',
        data=df,
        data_dict={'treatment': 'treated',
                  'date':'time',
                  'post':'post',
                  'unitid':'unit_id',
                  'outcome':'y'},
        pre_process_data=None,
        pre_treatment_window=None)


f3['results_df']
# ## Figure out the alpha and lambda values
# w=alpha_lambda.get_alpha_lambda(sc_dict['C_pre'])
# alpha_lambda_to_use = alpha_lambda.alpha_lambda_transform(w.x)
# ## Take the alpha and lambda values, and estimate mu and omega
# di_est = di.predict_mu_omega(sc_dict['T_pre'], sc_dict['C_pre'], alpha_lambda_to_use, 
#                              treatment_window_pre_treatment)
# di_output = di.sc_style_results(sc_dict['T_pre'], sc_dict['T_pst'],
#                     sc_dict['C_pre'], sc_dict['C_pst'],
#                         di_est['mu'],di_est['omega'])
# di_validation = sc.sc_validation(treatment_pre=sc_dict['T_pre'], 
#                  treatment_pst=sc_dict['T_pst'],
#                  control_pre=sc_dict['C_pre'],
#                  control_pst=sc_dict['C_pst'], 
#                  mu=di_est['mu'],
#                  omega=di_est['omega'],
#                  pre_treatment_window=treatment_window_pre_treatment)

# ak7 = cl.predict_mu_omega(sc_dict['T_pre'], sc_dict['C_pre'], treatment_window_pre_treatment)
# cl_output = di.sc_style_results(sc_dict['T_pre'], sc_dict['T_pst'],
#                     sc_dict['C_pre'], sc_dict['C_pst'],
#                     ak7['mu'], ak7['omega'])
# sc_validation = sc.sc_validation(treatment_pre=sc_dict['T_pre'], 
#                  treatment_pst=sc_dict['T_pst'],
#                  control_pre=sc_dict['C_pre'],
#                  control_pst=sc_dict['C_pst'], 
#                  mu=ak7['mu'],
#                  omega=ak7['omega'],
#                  pre_treatment_window=treatment_window_pre_treatment)



Using ADH


NameError: name 'collect_sc_outputs_aggregate' is not defined